In [17]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt

In [27]:
face_folder = "/home/miguel/Documents/sr/face recognition/lbph/"
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("/home/miguel/opencv-3.3.0/data/haarcascades_cuda/haarcascade_frontalface_default.xml")
subjects = ["", "Bronco"] 

In [40]:
def detect_face(img):
    #convert the test image to gray scale as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow: Haar classifier
    face_cascade = cv2.CascadeClassifier("/home/miguel/opencv-3.3.0/data/haarcascades_cuda/haarcascade_frontalface_default.xml")

    #let's detect multiscale images(some images may be closer to camera than others)
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    print(len(faces))

    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]

    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

In [4]:
face_folder = sorted(glob.glob(face_folder + "bronco/*.jpeg"))
face = []
label = []
for image in face_folder:
    face_img = cv2.imread(image)
    face_img, _ = detect_face(face_img)
    face.append(face_img)
    label.append(1)

1
1
1
1
1


In [5]:
print(len(face))
print(len(label))

5
5


In [30]:
recognizer.train(face, np.array(label))
recognizer.save('trainer.yml')

In [33]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
 
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
    
def predict(test_img):
    #make a copy of the image as we don't want to change original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    label, confidence= recognizer.predict(face)
    #get name of respective label returned by face recognizer
    label_text = subjects[label]

    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)

    return img

In [39]:
print("Predicting images...")

#load test images
test_img1 = cv2.imread("bronco1.jpeg")
test_img2 = cv2.imread("autos.jpeg")
 
#perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
print("Prediction complete")
 
#display both images
plt.imshow(predicted_img1)
plt.show()
plt.imshow(predicted_img2)
plt.show()
cv2.waitKey(0)
cv2.destroyAllWindows()

Predicting images...
1
0


error: /home/miguel/opencv-3.3.0/modules/core/src/matrix.cpp:310: error: (-215) s >= 0 in function setSize
